In [1]:
using Plots;plotly()

Plots.PlotlyBackend()

In [2]:
using Optim
using Zygote
using Meteor.Utility
using Meteor.ExactDiag

In [3]:
using LinearAlgebra
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
# LinearAlgebra.BLAS.set_num_threads(8)

4

In [4]:
using Base.Threads;nthreads()

1

In [5]:
using DataFrames;using CSV

In [6]:
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end

function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end

function eye(n)
    k::Array{Complex{Float64}}=zeros(n,n)
    for i in 1:n
        k[i,i]=1
    end
    return k    
end

σ₊=[0 0;1 0]
σ₋=[0 1;0 0]
σ_x=[0 1;1 0]
σ_y=[0 -1im;1im 0]
σ_z=[1 0;0 -1]

function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));

dim=10
ν=1
η=0.08
γ=0.1*ν
dot_num=30
E=eye(2*dim)
initial_state=kron([1.0 0;0 0],thermal_state(dim,1))
#########密度矩阵向量化，超算子维数变成n^2*n^2维,态矩阵厄米，只用存一半？意义不大
function dm2vec(dm)#态向量化
#julia里是按列reshape的,reshape成列向量
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,2*dim,:)
end

n=kron(eye(2),a₊(dim)*a₋(dim))
en=kron([0 0;0 1],eye(dim))
a=a₊(dim)+a₋(dim)
a4=kron([0 1;1 0],sin(η*a))

function diss()
    dtheta = 0.01
    x=zeros((2*dim)^2,(2*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η*theta*a)
        aup=kron([0 0;1 0],jump')
        adown=kron([0 1;0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end

a5=diss() 
# A*ρ*B形式，ρ(d*d)向量化之后，A*X*B的作用效果，变为矩阵kron(tranpose(B),A),,求单位相应ρ[i,j],,A第i列的元素放在第j列再去乘B第j行，
# A的i列乘B的j行，，，A[:,i]*B[j,:],,密度矩阵按列向量化，A*X*B变换后矩阵的第d*(j-1)+i列是reshape(A[:,i]*B[j,:],:,1),,

function standing_sideband_Lindbladian_vec(Ω,Δ)
    unitary::Matrix{ComplexF64}=kron(E,-im*(ν*n-Δ*en+Ω*a4))+kron(transpose(im*((ν*n-Δ*en+Ω*a4))),E)
    
    dissipation=a5
     
    return unitary+γ*dissipation
end

instate=dm2vec(initial_state);


In [7]:
function feval(x)
    f=exp(240*standing_sideband_Lindbladian_vec(x[1],x[2]))*instate
    return real(tr(n*vec2dm(f)))
end        
res0=optimize(feval,[0.575 -1.0],LBFGS())

 * Status: success

 * Candidate solution
    Final objective value:     3.862081e-03

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 1.11e-06 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.11e-06 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.32e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 3.43e-11 ≰ 0.0e+00
    |g(x)|                 = 2.36e-11 ≤ 1.0e-08

 * Work counters
    Seconds run:   9  (vs limit Inf)
    Iterations:    4
    f(x) calls:    14
    ∇f(x) calls:   14


In [8]:
Optim.minimizer(res0)

1×2 Array{Float64,2}:
 0.575971  -0.995437

In [9]:
Optim.minimum(res0)

0.003862081260974397

In [8]:
LinearAlgebra.BLAS.set_num_threads(8)
function testplot()
    ttot=500
    num=100
    ome=[0.1 0.2 0.3 0.4 0.5]
    state=instate
    x=[[real(tr(n*vec2dm(state)))] for i in 1:5]
    for i in 1:5
        for j in 1:num
            state= exp(ttot/num*standing_sideband_Lindbladian_vec(ome[i],-1))*state
            append!(x[i],real(tr(n*vec2dm(state))))  
        end
        state=instate
    end
    plot(ytick=0:0.2:1)
    plot!(0:ttot/num:ttot,x[1]);plot!(0:ttot/num:ttot,x[2]);plot!(0:ttot/num:ttot,x[3]);
    plot!(0:ttot/num:ttot,x[4]);plot!(0:ttot/num:ttot,x[5])
end 
testplot()

<!DOCTYPE html>
 
 
 Plots.jl

In [9]:
LinearAlgebra.BLAS.set_num_threads(1)
function sweep_detuning(ttot)
    initial_x=[0.2]
    sweep_num=90
    start=-1.4
    stop=-0.5
    d=zeros(sweep_num)
    sx=zeros(sweep_num)
    sn=zeros(sweep_num)
    @threads for i in 1:sweep_num
        print(i," ")
        m=start+(i-1)*(stop-start)/sweep_num
        function feval(x)
            f=exp(ttot*standing_sideband_Lindbladian_vec(x[1],m))*instate
            return real(tr(n*vec2dm(f)))
        end        
        
        a=optimize(feval,[0.0],[100.0],initial_x,Fminbox(LBFGS()))
        initial_x=Optim.minimizer(a) 
        sx[i]=initial_x[1]
        sn[i]=Optim.minimum(a)[1]
        d[i]=m
    end
    return sx,sn,d
end

sweep_detuning (generic function with 1 method)

In [10]:
res1=sweep_detuning(80);df1=DataFrame(delta=res1[3],omega=res1[1],n=res1[2]);CSV.write("sweep_detuning_80.csv", df1);
res2=sweep_detuning(160);df2=DataFrame(delta=res2[3],omega=res2[1],n=res2[2]);CSV.write("sweep_detuning_160.csv", df2);
res3=sweep_detuning(240);df3=DataFrame(delta=res3[3],omega=res3[1],n=res3[2]);CSV.write("sweep_detuning_240.csv", df3);

36 25 47 69 58 1 13 80 81 2 48 70 26 14 37 59 49 82 71 3 15 27 38 50 83 60 72 16 28 39 84 51 17 29 73 40 4 85 52 5 61 18 41 30 74 86 53 62 6 42 31 75 87 63 19 43 54 7 32 88 76 44 64 8 55 20 89 33 45 65 77 90 21 9 56 46 22 10 78 66 34 57 23 35 79 67 11 24 12 68 1 80 36 58 69 13 25 47 81 70 2 14 37 48 59 82 26 3 71 83 60 15 38 4 49 72 27 84 61 5 39 73 16 50 85 28 6 74 62 40 86 51 7 17 29 75 87 41 8 63 52 76 88 30 18 64 42 9 53 89 77 43 19 90 10 31 65 54 78 11 55 79 32 44 20 66 12 56 33 21 45 67 57 68 34 22 46 23 35 24 36 58 25 1 80 47 13 69 81 48 37 82 59 14 26 70 2 49 38 83 84 60 50 71 15 3 39 27 85 72 51 61 40 28 86 16 4 73 87 41 52 62 29 17 88 5 42 53 74 63 30 89 43 54 75 18 6 64 90 31 44 76 55 65 19 32 77 45 66 56 20 7 33 78 67 46 57 21 8 34 79 68 22 9 35 23 10 24 11 12 

In [14]:
data80=CSV.read("sweep_detuning_80.csv", DataFrame);
data160=CSV.read("sweep_detuning_160.csv", DataFrame);
data240=CSV.read("sweep_detuning_240.csv", DataFrame);
plot(xlabel="Δ/ν",xticks=-1.4:0.1:-0.5)
plot!(data80.delta, data80.n, label= "n80",color=:orange)
plot!(data80.delta, data80.omega,label= "omega80",color=:orange)
plot!(data160.delta, data160.n, label= "n160",color=:green)
plot!(data160.delta, data160.omega,label= "omega160",color=:green)
plot!(data240.delta, data240.n, label= "n240",color=:brown)
plot!(data240.delta, data240.omega,label= "omega240",color=:brown)

<!DOCTYPE html>
 
 
 Plots.jl

In [19]:
function stand_wave_side_band_cooling_model(d, Delta, nu, eta, omega, gamma)
    ps = nlevel(["g", "e"])
    pb = boson(d=d)
    
    xop = pb["a"] + pb["adag"]
    exp_xop = (exp(im*eta*xop) - exp(-im*eta*xop)) / (2*im)
    
    model = build_model([ps, pb], isunitary=false)

    add_unitary!(model, (1,), ("e<-e",), coeff=-Delta)
    add_unitary!(model, (2,), ("n",), coeff=nu)
    add_unitary!(model, (1, 2), ("e<-g", exp_xop), coeff=omega)
    add_unitary!(model, (1, 2), ("g<-e", exp_xop'), coeff=omega)
    
    dtheta = 0.01
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*eta*theta*xop)
        add_dissipation!(model, (1, 2), ("g<-e", jump), coeff=gamma/2)
    end
#     add_dissipation!(model, (1,), ("g<-e",), coeff=gamma/2)
    add_observer!(model, (1,), ("e<-e",), name="e")
    add_observer!(model, (2,), ("n",), name="n")

    model.state=kron(thermal_state(d,4),[1.0 0;0 0])
    return model
end

function stand_wave_side_band_cooling_b(d, Delta, nu, eta, omega, gamma, t)
    model = stand_wave_side_band_cooling_model(d, Delta, nu, eta, omega, gamma)
    ham, observer, state = model.h, model.observer, model.state
    ham = matrix(immrep(ham))
    L = size(state, 1)
    observer = immrep(observer)
    obs = measure(observer, state)
    obs = Observables(obs)
    
    state = reshape(state, L * L)
    mt = t/100
    nt = round(Int, t / mt)
    for i in 1:nt
        state = evolve(ham, mt, state, ishermitian=false, driver=:krylov)
        append!(obs, measure(observer, reshape(state, L, L)))
    end
    return todict(obs)
end

stand_wave_side_band_cooling_b (generic function with 1 method)

In [20]:
# compute the steady state
function steady_state(d, Delta, nu, eta, omega, gamma)
    model = stand_wave_side_band_cooling_model(d, Delta, nu, eta, omega, gamma)
    results = steady_state!(model)
    return todict(results.observables)
end
LinearAlgebra.BLAS.set_num_threads(1)
function sweep_steady_state(result)
    sn=zeros(length(result.delta))
    sin=zeros(length(result.delta))
    @threads for i in 1:length(result.delta)
        print(i," ")
        a=steady_state(dim, result.delta[i], ν, η, result.omega[i], γ)
        sn[i]=a["n[2]"][1]
        sin[i]=a["e[1]"][1]
    end
    return sn,sin
end

sweep_steady_state (generic function with 1 method)

In [21]:
resu=CSV.read("sweep_detuning_240.csv", DataFrame);

In [22]:
s=sweep_steady_state(resu);ss=DataFrame(delta=resu.delta,omega=resu.omega,nbose=s[1],pe=s[2]);CSV.write("steady240.csv", ss);

1 69 58 36 80 13 25 47 26 70 48 27 37 14 71 59 2 81 28 49 60 72 38 15 73 50 3 61 29 51 82 74 39 52 4 62 53 30 75 40 83 16 63 54 55 64 76 41 31 56 65 84 42 66 17 57 77 43 32 5 78 67 44 68 85 33 79 45 6 34 46 18 86 35 7 19 87 8 9 88 20 21 10 89 11 22 90 23 24 12 

In [25]:
data_steady240=CSV.read("steady240.csv", DataFrame);
data_240=CSV.read("sweep_detuning_240.csv", DataFrame);
plot(title="ttot=240 steady n")
plot!(data_240.delta,data_240.omega,label= "omega240",color=:green)
plot!(data_240.delta,data_240.n,label= "n240",color=:red)
plot!(data_240.delta,data_steady240.nbose,label= "steady_n240",color=:brown)
# plot!(data_240.delta,data_steady240.pe,label= "steady_e_femi240",color=:black)

<!DOCTYPE html>
 
 
 Plots.jl

In [34]:
LinearAlgebra.BLAS.set_num_threads(1)
function stand_wave_side_band_cooling_c( Delta,omega, t)
    model = stand_wave_side_band_cooling_model(dim, Delta, ν, η, omega, γ)
    ham, observer, state = model.h, model.observer, model.state
    ham = matrix(immrep(ham))
    L = size(state, 1)
    observer = immrep(observer)
    obs = measure(observer, state)
    obs = Observables(obs)
    
    state = reshape(state, L * L)

    state = evolve(ham, t, state, ishermitian=false, driver=:krylov)
    append!(obs, measure(observer, reshape(state, L, L)))
    return todict(obs)
end

function nt()
    ttot=240
    space=0.01
    space2=2.5/0.9*space
    s_delta=-1.4:space: -0.5
    s_omega=0.5:space2:3
    n = zeros(length(s_delta),length(s_omega))
    e = zeros(length(s_delta),length(s_omega))
    datane=DataFrame(delta=s_delta,omega=s_omega)
    @threads for omega in s_omega
        for delta in s_delta
            n[convert(Int64,round((delta-s_delta[1])/space+1)),convert(Int64,round((omega-s_omega[1])/space2+1))]=stand_wave_side_band_cooling_c(delta,omega, ttot)["n[2]"][2]
            e[convert(Int64,round((delta-s_delta[1])/space+1)),convert(Int64,round((omega-s_omega[1])/space2+1))]=stand_wave_side_band_cooling_c(delta,omega, ttot)["e[1]"][2]
        end
        print(omega," ")
    end
    datane=hcat(datane,DataFrame(hcat(n,e),:auto))
    CSV.write("delta_omega_sweep.csv", datane)
    return n,e
end

nt (generic function with 1 method)

In [35]:
# @time omde=nt()

0.5 0.5277777777777778 0.5555555555555556 0.5833333333333334 0.6111111111111112 0.6388888888888888 0.6666666666666666 0.6944444444444444 0.7222222222222222 0.75 0.7777777777777778 0.8055555555555556 0.8333333333333334 0.8611111111111112 0.8888888888888888 0.9166666666666666 0.9444444444444444 0.9722222222222222 1.0 1.0277777777777777 1.0555555555555556 1.0833333333333333 1.1111111111111112 1.1388888888888888 1.1666666666666667 1.1944444444444444 1.2222222222222223 1.25 1.2777777777777777 1.3055555555555556 1.3333333333333333 1.3611111111111112 1.3888888888888888 1.4166666666666667 1.4444444444444444 1.4722222222222223 1.5 1.5277777777777777 1.5555555555555556 1.5833333333333333 1.6111111111111112 1.6388888888888888 1.6666666666666667 1.6944444444444444 1.7222222222222223 1.75 1.7777777777777777 1.8055555555555556 1.8333333333333333 1.8611111111111112 1.8888888888888888 1.9166666666666667 1.9444444444444444 1.9722222222222223 2.0 2.0277777777777777 2.0555555555555554 2.0833333333333335 

([1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0], [2.0 2.0 … 2.0 2.0; 2.0 2.0 … 2.0 2.0; … ; 2.0 2.0 … 2.0 2.0; 2.0 2.0 … 2.0 2.0])

In [8]:
data3d=CSV.read("delta_omega_sweep.csv", DataFrame)
nesurf=Array(data3d[:,Between(:x1, end)])
row=convert(Int64,size(nesurf)[2]/2)
nsurf=nesurf[:,1:row];
esurf=nesurf[:,row+1:end];
surface(data3d.omega,data3d.delta,nsurf,xlabel="omega",ylabel="Δ",zlabel="n_t",xticks=0.5:0.5:3,yticks=-1.4:0.15:-0.5,zticks=0:0.3:3)

<!DOCTYPE html>
 
 
 Plots.jl